In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
import time
import os
import numpy as np
from pathlib import Path
from PIL import Image
import helper
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.image import imread
from torchvision import datasets
from torchvision import datasets, transforms, models
from torch import nn, optim, Tensor
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from torch.utils.data import DataLoader, Dataset, TensorDataset
import random
seed=42
torch.manual_seed(14)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

In [ ]:
class RGBCloudDataset (Dataset):
    def __init__(self, red_dir, blue_dir, green_dir, gt_dir):
        

        # Listing subdirectories
        # Loop through the files in red folder  
        # and combine, into a dictionary, the other bands
        
        self.files = [self.combine_files(f, green_dir, blue_dir,gt_dir) 
                      for f in red_dir.iterdir() if not f.is_dir()]
        
        random.seed (seed)
        self.files = random.sample (self.files, k= 10000)   
        
    def combine_files(self, red_file: Path, green_dir, blue_dir,  gt_dir):
        
        files = {'red': red_file, 
                 'green':green_dir/red_file.name.replace('red', 'green'),
                 'blue': blue_dir/red_file.name.replace('red', 'blue'), 
                 'gt': gt_dir/red_file.name.replace('red', 'gt')}

        return files
    
    
    
    def OpenAsArray(self, idx, invert=False):
        
        raw_rgb=np.stack([np.array(Image.open(self.files[idx]['red'])),
                          np.array(Image.open(self.files[idx]['green'])),
                          np.array(Image.open(self.files[idx]['blue']))], axis = 2)
     

        if invert:
            raw_rgb = raw_rgb.transpose((2, 0, 1))
    
    
        return (raw_rgb / np.iinfo(raw_rgb.dtype).max)
    
    
    
    
    def OpenMask(self, idx, add_dims=False):
        
        raw_mask=np.array(Image.open(self.files[idx]['gt']))
        raw_mask = np.where(raw_mask==255, 1, 0)
        
        
        return np.expand_dims(raw_mask, 0) if add_dims else raw_mask



        
    def __len__(self):
        
        return len(self.files)
    
    
    
    def __getitem__(self, idx):
        
        x = torch.tensor(self.OpenAsArray(idx, invert=True), dtype=torch.float32)
        y = torch.tensor(self.OpenMask(idx, add_dims=False), dtype=torch.int64)
        
        return x, y
    
    
    
    def open_as_pil(self, idx):
        
        arr = 256 * self.OpenAsArray(idx)
        
        return Image.fromarray(arr.astype(np.uint8), 'RGB')  
    
    
    
    def __repr__(self):
        
        s = 'Dataset class with {} files'.format(self.__len__())

        return s

In [ ]:
base_path = Path('../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud')
red_dir   = base_path/'train_red_additional_to38cloud'
blue_dir  = base_path/'train_blue_additional_to38cloud'
green_dir = base_path/'train_green_additional_to38cloud'
nir_dir   = base_path/'train_nir_additional_to38cloud'
gt_dir    = base_path/'train_gt_additional_to38cloud'

RGBdata = RGBCloudDataset(red_dir, blue_dir, green_dir, gt_dir)  

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,4, figsize = (10,9))
plt.axis('off')
red=np.array(Image.open("../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud/train_red_additional_to38cloud/red_patch_100_5_by_12_LC08_L1TP_047011_20160920_20170221_01_T1.TIF"))
green=np.array(Image.open("../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud/train_green_additional_to38cloud/green_patch_100_5_by_12_LC08_L1TP_047011_20160920_20170221_01_T1.TIF"))
blue=np.array(Image.open("../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud/train_blue_additional_to38cloud/blue_patch_100_5_by_12_LC08_L1TP_047011_20160920_20170221_01_T1.TIF"))
nir=np.array(Image.open("../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud/train_nir_additional_to38cloud/nir_patch_100_5_by_12_LC08_L1TP_047011_20160920_20170221_01_T1.TIF"))
plt.axis('off')
ax[0].imshow(red, cmap='gray')
ax[1].imshow(green, cmap='gray')
ax[2].imshow(blue, cmap='gray')
ax[3].imshow(nir, cmap='gray')

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,1, figsize = (5,5))
nir=np.array(Image.open("../input/95cloud-cloud-segmentation-on-satellite-images/95-cloud_training_only_additional_to38-cloud/train_nir_additional_to38cloud/nir_patch_100_5_by_12_LC08_L1TP_047011_20160920_20170221_01_T1.TIF"))
ax.imshow(nir, cmap='gray')
plt.axis('off')
plt.savefig("nir_channel.png", bbox_inches='tight')

In [ ]:
x, y =RGBdata[1000]
x.shape, y. shape

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,2, figsize = (10,9))
ax[0].imshow(RGBdata.OpenAsArray(89))
ax[1].imshow(RGBdata.OpenMask(89))
plt.axis('off')
plt.savefig("Original_samples7.png", bbox_inches='tight')

In [ ]:
# splitting the data into train, validation, and test datasets

train_size = int(0.75 * len(RGBdata))
valid_size = int(0.15 * len(RGBdata))
test_size  = len(RGBdata) - train_size - valid_size
remaining_size = len(RGBdata) - train_size 

RGBtrain_dataset, RGBremaining_dataset = torch.utils.data.random_split(RGBdata, [train_size, remaining_size])
RGBvalid_dataset, RGBtest_dataset      = torch.utils.data.random_split(RGBremaining_dataset, [valid_size, test_size])


print('\t\t\tDataset')
print("Train data: \t\t{}".format(len(RGBtrain_dataset)),
      "\nValidation data: \t{}".format(len(RGBvalid_dataset)),
     "\nTest data: \t\t{}".format(len(RGBtest_dataset)))



RGBtrain_loader = DataLoader(RGBtrain_dataset, batch_size=32, shuffle=True, num_workers=2)
RGBvalid_loader = DataLoader(RGBvalid_dataset, batch_size=32, shuffle=True, num_workers=2)
RGBtest_loader  = DataLoader(RGBtest_dataset , batch_size=32, shuffle=False, num_workers=2)

data_iter = iter(RGBvalid_loader)
rgb_img, mask = next(data_iter)

print('\n')
print('Raw RGB image shape on batch size = {}'.format(rgb_img.size()))
print('Cloud Mask shape on batch size    = {}'.format(mask.size()))

In [ ]:
def batch_to_img(xb, idx):
    img = np.array(xb[idx,0:3])
    return img.transpose((1,2,0))

def predb_to_mask(predb, idx):
    p = torch.functional.F.softmax(predb[idx], 0)
    return p.argmax(0).cpu()

In [ ]:
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.resnet34(pretrained=True)
        # Replace the classifier with an upsampler
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False))))
    
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[:8].parameters():
            param.require_grad = False
        for param in self.modified_network[8].parameters():
            param.require_grad = True
        for param in self.modified_network[9].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
class ResNet34_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.resnet34(pretrained=False)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1, bias=False),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1, bias=False),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False))))
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)

In [ ]:
class Vgg16(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=True)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 

    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[0].parameters():
            param.require_grad = False
        for param in self.modified_network[1:].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
class Vgg16_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=False)
        # Replace the classifier with an upsampling block
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)

In [ ]:
PATH = "../input/untrained-vgg16-dicebce/Untrained_Vgg16_DiceBCE_lr4.pt"


model = Vgg16_np()
map_location=torch.device('cpu')
model.load_state_dict(torch.load(PATH,map_location=torch.device('cpu')),strict=False)
model.eval()

In [ ]:
xb, yb = next(iter(RGBtest_loader))

with torch.no_grad():
    predb = model(xb.to(device))

predb.shape

In [ ]:
bs = 32
fig, ax = plt.subplots(bs,3, figsize=(15,bs*5))
for i in range(bs):
    ax[i,0].imshow(batch_to_img(xb,i))
    ax[i,1].imshow(yb[i])
    ax[i,2].imshow(predb_to_mask(predb, i))

In [ ]:
fig.savefig('./bs32_untrained_vgg16_DiceBCE.png') 

In [ ]:
# Evaluation Metrics

class Evaluation_Metrics(nn.Module):
    def __init__(self):
        super(Evaluation_Metrics, self).__init__()

    def forward(self, prediction, gt, smooth=1):
        
        pred = torch.round(prediction.softmax(dim=1)[:, 1])

        # true positives, false positives, true negatives, false negatives
        TP = torch.sum(pred * gt)
        FP = torch.sum(pred * (1-gt))
        TN = torch.sum((1-pred) * (1-gt))
        FN = torch.sum((1-pred) * gt)
    
    
        # Dice_Score/F1_Score
        Dice_Score = (2 * TP + smooth)/(2*TP + FP + FN  + smooth)
    
        # Jaccard Coefficient: Intersection over Union
        IoU = (TP + smooth)/(TP + FP + FN + smooth)
    
        # Recall
        Recall= (TP + smooth)/(TP + FN + smooth)
    
        # Precision
        Precision = (TP + smooth)/(TP + FP + smooth)

    
        return {'Dice_Score/F1_Score':Dice_Score, 'IoU':IoU, 'Recall': Recall, 'Precision': Precision}


# Dice Loss function
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self,prediction, gt, smooth=1):
        
        # Softmax to get probabilities beteen 0 and 1
        # Transform the prediction tensor of shape (N, C, H, W) --> tensor of shape (N, H, W)
        pred = prediction.softmax(dim=1)[:, 1]
        
        #flatten label and prediction tensors
        pred = pred.contiguous().view(-1)
        gt = gt.contiguous().view(-1).to(torch.float32)
        
        intersection = (pred * gt).sum()                            
        dice_loss =(2.*intersection + smooth)/(pred.sum() + gt.sum() + smooth)  
        
        return -torch.log(dice_loss)
    

#Binary cross-entropy (BCE)-Dice loss function
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()

    def forward(self, prediction, gt, smooth=1):
        
        # Softmax to get probabilities beteen 0 and 1
        # Transform the prediction tensor of shape (N, C, H, W) --> tensor of shape (N, H, W)
        pred = prediction.softmax(dim=1)[:, 1]
        
        #flatten label and prediction tensors
        pred = pred.contiguous().view(-1)
        gt = gt.contiguous().view(-1).to(torch.float32)
        
        intersection = (pred * gt).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(pred.sum() + gt.sum() + smooth)  
        BCE = F.binary_cross_entropy(pred, gt, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [ ]:
# Set the evaluation metrics and the loss function
Evaluation = Evaluation_Metrics()
loss_fn    = DiceBCELoss()
logDice_loss = DiceLoss()

In [ ]:
test_loss = 0.0
test_DSC  = 0.0
test_IoU  = 0.0
test_recall  = 0.0
test_precision  = 0.0


# iterate over test data
for x, y in RGBtest_loader:
    x = x.to(device, dtype=torch.float)
    y = y.to(device)
    
    
    with torch.no_grad():
        model.eval()
        outputs = model(x)
        loss    = logDice_loss(outputs, y)
                
        # stats - whatever is the phase  
        acc = Evaluation(outputs, y)
        test_DSC        += acc['Dice_Score/F1_Score'].item()* RGBtest_loader.batch_size
        test_IoU        += acc['IoU'].item()* RGBtest_loader.batch_size
        test_recall     += acc['Recall'].item()* RGBtest_loader.batch_size
        test_precision  += acc['Precision'].item()* RGBtest_loader.batch_size    
        test_loss += loss.item() * RGBtest_loader.batch_size
                    
epoch_loss       = test_loss/ len(RGBtest_loader.dataset)
epoch_DSC        = test_DSC / len(RGBtest_loader.dataset)
epoch_IoU        = test_IoU / len(RGBtest_loader.dataset)
epoch_Recall     = test_recall / len(RGBtest_loader.dataset)
epoch_Precision  = test_precision / len(RGBtest_loader.dataset)
            
            
print('Dice Loss: {:.3f}\tDice Coefficient: {:.3f}\tJaccard Coefficient: {:.3f}\tPrecision: {:.3f}\tRecall: {:.3f}'.format(epoch_loss, epoch_DSC, epoch_IoU, epoch_Precision, epoch_Recall))
print()

In [ ]:
network = models.resnet34(pretrained=True)
network 

## **Model: Pretrained Vgg16**

In [ ]:
class Vgg16(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=True)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 

    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[0].parameters():
            param.require_grad = False
        for param in self.modified_network[1:].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
# Evaluation Metrics

class Evaluation_Metrics(nn.Module):
    def __init__(self):
        super(Evaluation_Metrics, self).__init__()

    def forward(self, prediction, gt, smooth=1):
        
        pred = torch.round(prediction.softmax(dim=1)[:, 1])

        # true positives, false positives, true negatives, false negatives
        TP = torch.sum(pred * gt)
        FP = torch.sum(pred * (1-gt))
        TN = torch.sum((1-pred) * (1-gt))
        FN = torch.sum((1-pred) * gt)
    
    
        # Dice_Score/F1_Score
        Dice_Score = (2 * TP + smooth)/(2*TP + FP + FN  + smooth)
    
        # Jaccard Coefficient: Intersection over Union
        IoU = (TP + smooth)/(TP + FP + FN + smooth)
    
        # Recall
        Recall= (TP + smooth)/(TP + FN + smooth)
    
        # Precision
        Precision = (TP + smooth)/(TP + FP + smooth)

    
        return {'Dice_Score/F1_Score':Dice_Score, 'IoU':IoU, 'Recall': Recall, 'Precision': Precision}


# Dice Loss function
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()

    def forward(self, prediction, gt, smooth=1):
        
        # Softmax to get probabilities beteen 0 and 1
        # Transform the prediction tensor of shape (N, C, H, W) --> tensor of shape (N, H, W)
        pred = prediction.softmax(dim=1)[:, 1]
        
        #flatten label and prediction tensors
        pred = pred.contiguous().view(-1)
        gt = gt.contiguous().view(-1).to(torch.float32)
        
        intersection = (pred * gt).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(pred.sum() + gt.sum() + smooth)  
        BCE = F.binary_cross_entropy(pred, gt, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [ ]:
# Use GPU if it is available
model = Vgg16().to(device)

#checking if we have the correct dimensions for our (3,384,384) images
#summary(Vgg16, (3, 384, 384))   

# checking if the network works, using one batch of the training set
rgb_img, mask = next(iter(RGBtrain_loader))
rgb_img, mask = rgb_img.to(device, dtype=torch.float), mask.to(device) 
output = model (rgb_img).to(device)
output.shape, mask.shape 

In [ ]:
# Set the evaluation metrics and the loss function
Evaluation = Evaluation_Metrics()
loss_fn    = DiceBCELoss()

# Set the optimizer 
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def plot_DSC(train_DSC, valid_DSC):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_DSC, '-bx')
    plt.plot(valid_DSC, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('Dice Coefficient')
    plt.title('Dice Coefficient vs. No. of epochs');
    
def plot_losses(train_losses, valid_losses):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_losses, '-bx')
    plt.plot(valid_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('DiceBCE loss')
    plt.legend(['Training', 'Validation'])
    plt.title('DiceBCE Loss vs. No. of epochs');

In [ ]:
def train(model, train_dl, valid_dl, loss_fn, optimizer, Evaluation, epochs, save_path):
    
    start = time.time()
    min_valid_epoch_loss = np.inf
    best_DSC = 0.0
    train_losses, valid_losses = [], []
    train_DSC, train_IoU, train_recall, train_precision = [], [], [], [] 
    valid_DSC, valid_IoU, valid_recall, valid_precision = [], [], [], [] 

    for e in range(epochs):
        print('Epoch  {}/{}'.format(e, epochs-1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to train mode
                dataloader = train_dl
            else:
                model.eval()  # Set model to evaluate mode
                dataloader = valid_dl

            running_loss = 0.0
            running_DSC  = 0.0
            running_IoU  = 0.0
            running_recall  = 0.0
            running_precision  = 0.0


            # iterate over data
            for x, y in dataloader:
                x = x.to(device, dtype=torch.float)
                y = y.to(device)
                

                # forward pass 
                if phase == 'train':
                    # zero the gradients
                    optimizer.zero_grad()
                    outputs = model(x)
                    loss    = loss_fn(outputs, y)
                    # backward + optimize only if in training phase
                    # the backward pass frees the graph memory, so there is no 
                    # need for torch.no_grad in this training pass
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()

                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss    = loss_fn(outputs, y)

                # stats - whatever is the phase  
                acc = Evaluation(outputs, y)
                running_DSC        += acc['Dice_Score/F1_Score'].item()* dataloader.batch_size
                running_IoU        += acc['IoU'].item()* dataloader.batch_size
                running_recall     += acc['Recall'].item()* dataloader.batch_size
                running_precision  += acc['Precision'].item()* dataloader.batch_size    
                
                running_loss += loss.item() * dataloader.batch_size
                    
            epoch_loss       = running_loss/ len(dataloader.dataset)
            epoch_DSC        = running_DSC / len(dataloader.dataset)
            epoch_IoU        = running_IoU / len(dataloader.dataset)
            epoch_Recall     = running_recall / len(dataloader.dataset)
            epoch_Precision  = running_precision / len(dataloader.dataset)
            

            print('{}\nDice Loss: {:.3f}\tDice Coefficient: {:.3f}\tJaccard Coefficient: {:.3f}\tPrecision: {:.3f}\tRecall: {:.3f}'.format( phase, epoch_loss, epoch_DSC, epoch_IoU, epoch_Precision, epoch_Recall))
            print()

            train_losses.append(epoch_loss), train_DSC.append(epoch_DSC),train_IoU.append(epoch_IoU),train_recall.append(epoch_Recall),train_precision.append(epoch_Precision) if phase=='train' else valid_losses.append(epoch_loss), valid_DSC.append(epoch_DSC),valid_IoU.append(epoch_IoU),valid_recall.append(epoch_Recall),valid_precision.append(epoch_Precision)

            
             
             # save model if validation loss has decreased
            if phase == 'valid':
                if epoch_loss <= min_valid_epoch_loss:
                    print('Validation Dice loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(
                        min_valid_epoch_loss, epoch_loss)) 
                    print('Best Validation Dice Score ({:.3f} --> {:.3f}).'.format(
                        best_DSC, epoch_DSC))
                    torch.save(model.state_dict(), save_path)
                    min_valid_epoch_loss = epoch_loss
                    best_DSC = epoch_DSC
            

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    
    return train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC

In [ ]:
model.freeze()
Pretrained_Vgg16 = './Pretrained_Vgg16_DiceScore_lr4_bs32.pt'
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC = train(model, RGBtrain_loader, RGBvalid_loader, loss_fn, optimizer, Evaluation, epochs=40, save_path = Pretrained_Vgg16)

In [ ]:
plot_DSC(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Pretrained_Vgg16_bs32.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Pretrained_Vgg16_bs32.png')

In [ ]:
# Dice Loss function
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self,prediction, gt, smooth=1):
        
        # Softmax to get probabilities beteen 0 and 1
        # Transform the prediction tensor of shape (N, C, H, W) --> tensor of shape (N, H, W)
        pred = prediction.softmax(dim=1)[:, 1]
        
        #flatten label and prediction tensors
        pred = pred.contiguous().view(-1)
        gt = gt.contiguous().view(-1).to(torch.float32)
        
        intersection = (pred * gt).sum()                            
        dice_loss =(2.*intersection + smooth)/(pred.sum() + gt.sum() + smooth)  
        
        return -torch.log(dice_loss)

logDice_loss = DiceLoss()


def plot_DSC(train_DSC, valid_DSC):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_DSC, '-bx')
    plt.plot(valid_DSC, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('Dice Coefficient')
    plt.title('Dice Coefficient vs. No. of epochs');
    
def plot_losses(train_losses, valid_losses):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_losses, '-bx')
    plt.plot(valid_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('- Log Dice loss')
    plt.legend(['Training', 'Validation'])
    plt.title('- Log Dice Loss vs. No. of epochs');

In [ ]:
model2 = Vgg16().to(device)
model2.freeze()
# Set the optimizer 
optimizer = optim.Adam(model2.parameters(), lr=0.0001) 
pretrained_vgg16_path = './Vgg16_logdice_lr4_bs32.pt'  
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC= train(model2, RGBtrain_loader, RGBvalid_loader, logDice_loss, optimizer,Evaluation, epochs=40, save_path = pretrained_vgg16_path)

In [ ]:
plot_DSC(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Vgg16_logdice_lr4_32bs.png')   

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Vgg16_logdice_lr4_32bs.png')  

## **Model: Untrained Vgg16**

In [ ]:
class Vgg16_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.vgg16(pretrained=False)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,512, kernel_size = 1),
                                                     nn.BatchNorm2d(512),
                                                     nn.ReLU(),
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.BatchNorm2d(2),
                                                     nn.ReLU(),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)

In [ ]:
# Use GPU if it is available 
model_np = Vgg16_np().to(device)

# Set the optimizer 
optimizer = optim.Adam(model.parameters(), lr=0.001)

Untrained_Vgg16 = './Untrained_Vgg16_DiceScore.pt'
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC = train(model_np,RGBtrain_loader,RGBvalid_loader, loss_fn, optimizer, Evaluation, epochs=25, save_path = Untrained_Vgg16)

In [ ]:
plot_DSC(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Untrained_Vgg16.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Untrained_Vgg16.png')

## **Model: Pretrained Resnet34**

In [ ]:
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        network = models.resnet34(pretrained=True)
        # Replace the classifier
        self.modified_network = nn.Sequential(*list((*list(network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.modified_network(xb)
    
    def freeze(self):
        # To freeze the CONV layers
        for param in self.modified_network[:8].parameters():
            param.require_grad = False
        for param in self.modified_network[8].parameters():
            param.require_grad = True
        for param in self.modified_network[9].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.modified_network.parameters():
            param.require_grad = True

In [ ]:
# Use GPU of available
ResNet_model2 = ResNet34().to(device)

# checking if the network works, using one batch of the training set
img, mask = next(iter(RGBtrain_loader))
img, mask = rgb_img.to(device, dtype=torch.float), mask.to(device) 
output = ResNet_model (img).to(device)
output.shape, mask.shape 

In [ ]:
# Set up cutom optimizer with weight decay
optimizer = optim.Adam(ResNet_model2.parameters(), lr=0.0001)

ResNet_model2.freeze()
Pretrained_ResNet34 = './Pretrained_ResNet34_DiceScore_lr4_bs32.pt'
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC = train(ResNet_model2,RGBtrain_loader, RGBvalid_loader, loss_fn, optimizer, Evaluation, epochs=40, save_path = Pretrained_ResNet34) 

In [ ]:
def plot_DSC(train_DSC, valid_DSC):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_DSC, '-bx')
    plt.plot(valid_DSC, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('Dice Coefficient')
    plt.title('Dice Coefficient vs. No. of epochs');
    
def plot_losses(train_losses, valid_losses):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(train_losses, '-bx')
    plt.plot(valid_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('DiceBCE loss')
    plt.legend(['Training', 'Validation'])
    plt.title('DiceBCE Loss vs. No. of epochs');

In [ ]:
plot_DSC(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Pretrained_ResNet34_lr4_bs32.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Pretrained_ResNet34_lr4_bs32.png')

In [ ]:
model3 = ResNet34().to(device)
model3.freeze()
# Set the optimizer 
optimizer = optim.Adam(model3.parameters(), lr=0.0001) 
pretrained_resnet34_path = './Resnet34_logdice_lr4_bs32.pt'  
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC= train(model3, RGBtrain_loader, RGBvalid_loader, logDice_loss, optimizer,Evaluation, epochs=40, save_path = pretrained_resnet34_path)

In [ ]:
plot_DSC(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_resnet34_logdice_lr4_32bs.png')   

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Pretrained_ResNet34_logdice_lr4_bs32.png')

## **Model: Untrained Resnet34**

In [ ]:
class ResNet34_np(nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=False)
        # Replace the classifier
        self.network = nn.Sequential(*list((*list(self.network.children())[:-2],
                                                     nn.Conv2d(512,2, kernel_size = 1),
                                                     nn.Upsample(size=(384,384), mode='bilinear', align_corners=False)))) 
    
    
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
# Use GPU if available
Resnet_np = ResNet34_np().to(device)

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(Resnet_np.parameters(), lr=0.001)

Untrained_ResNet34 = './Untrained_ResNet34_DiceScore.pt'
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC = train(Resnet_np,RGBtrain_loader, RGBvalid_loader, loss_fn, optimizer, Evaluation, epochs=25, save_path = Untrained_ResNet34)

In [ ]:
plot_accuracies(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Untrained_ResNet34.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Untrained_ResNet34.png')

**A simple Unet**

In [ ]:
class CloudDataset (Dataset):
    def __init__(self, red_dir, blue_dir, green_dir, nir_dir, gt_dir):
        

        # Listing subdirectories
        # Loop through the files in red folder  
        # and combine, into a dictionary, the other bands
        
        self.files = [self.combine_files(f, green_dir, blue_dir, nir_dir, gt_dir) 
                      for f in red_dir.iterdir() if not f.is_dir()]
        
                
        
    def combine_files(self, red_file: Path, green_dir, blue_dir, nir_dir, gt_dir):
        
        files = {'red': red_file, 
                 'green':green_dir/red_file.name.replace('red', 'green'),
                 'blue': blue_dir/red_file.name.replace('red', 'blue'), 
                 'nir': nir_dir/red_file.name.replace('red', 'nir'),
                 'gt': gt_dir/red_file.name.replace('red', 'gt')}

        return files
    
    
    
    def OpenAsArray(self, idx, invert=False, include_nir=False):
        
        raw_rgb=np.stack([np.array(Image.open(self.files[idx]['red'])),
                          np.array(Image.open(self.files[idx]['green'])),
                          np.array(Image.open(self.files[idx]['blue']))], axis = 2)
     
     
        if include_nir:
            nir = np.expand_dims(np.array(Image.open(self.files[idx]['nir'])), axis = 2)
            raw_rgb = np.concatenate([raw_rgb, nir], axis = 2) 
                          
        if invert:
            raw_rgb = raw_rgb.transpose((2, 0, 1))
    
    
        return (raw_rgb / np.iinfo(raw_rgb.dtype).max)
    
    
    
    
    def OpenMask(self, idx, add_dims=False):
        
        raw_mask=np.array(Image.open(self.files[idx]['gt']))
        raw_mask = np.where(raw_mask==255, 1, 0)
        
        
        return np.expand_dims(raw_mask, 0) if add_dims else raw_mask



        
    def __len__(self):
        
        return len(self.files)
    
    
    
    def __getitem__(self, idx):
        
        x = torch.tensor(self.OpenAsArray(idx, invert=True, include_nir=True), dtype=torch.float32)
        y = torch.tensor(self.OpenMask(idx, add_dims=False), dtype=torch.int64)
        
        return x, y
    
    
    
    def open_as_pil(self, idx):
        
        arr = 256 * self.OpenAsArray(idx)
        
        return Image.fromarray(arr.astype(np.uint8), 'RGB')  
    
    
    
    def __repr__(self):
        
        s = 'Dataset class with {} files'.format(self.__len__())

        return s

In [ ]:
data = CloudDataset(red_dir, blue_dir, green_dir, nir_dir, gt_dir) 

# splitting the data into train, validation, and test datasets

train_size = int(0.75 * len(data))
valid_size = int(0.15 * len(data))
test_size  = len(data) - train_size - valid_size
remaining_size = len(data) - train_size 

train_dataset, remaining_dataset = torch.utils.data.random_split(data, [train_size, remaining_size])
valid_dataset, test_dataset      = torch.utils.data.random_split(remaining_dataset, [valid_size, test_size])


print('\t\t\tDataset')
print("Train data: \t\t{}".format(len(train_dataset)),
      "\nValidation data: \t{}".format(len(valid_dataset)),
     "\nTest data: \t\t{}".format(len(test_dataset)))



train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=12, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset , batch_size=12, shuffle=True, num_workers=2)

data_iter = iter(valid_loader)
rgb_img, mask = next(data_iter)

print('\n')
print('Raw RGB image shape on batch size = {}'.format(rgb_img.size()))
print('Cloud Mask shape on batch size    = {}'.format(mask.size()))

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
            
        # downsampling part
        self.DownConv1 = self.ContractBlock(in_channels, 32, 7, 3)
        self.DownConv2 = self.ContractBlock(32, 64, 3, 1)
        self.DownConv3 = self.ContractBlock(64, 128, 3, 1)
            
        # upsampling part
        self.UpConv3 = self.ExpandBlock(128, 64, 3, 1)
        self.UpConv2 = self.ExpandBlock(64*2, 32, 3, 1)
        self.UpConv1 = self.ExpandBlock(32*2, out_channels, 3, 1)
        
    def __call__(self, x):
         
        DownConv1 = self.DownConv1(x)
        DownConv2 = self.DownConv2(DownConv1) 
        DownConv3 = self.DownConv3(DownConv2)   
        UpConv3   = self.UpConv3 (DownConv3)
        UpConv2   = self.UpConv2 (torch.cat([UpConv3, DownConv2], 1))
        UpConv1   = self.UpConv1 (torch.cat([UpConv2, DownConv1], 1))
        
        return UpConv1
        
        
    def ContractBlock(self, in_channels, out_channels, kernel_size, padding):
        
        contract = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
        return contract



    def ExpandBlock(self, in_channels, out_channels, kernel_size, padding):
        
        expand = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1) )
    
        return expand

In [ ]:
# checking if the network works, using one batch of the training set

Unet_model = UNet(4, 2)

data_iter = iter(train_loader)
img, mask = next(data_iter)

output = Unet_model (img)
output.shape

In [ ]:
# Use GPU if available
Unet_model.to(device)

# Set up cutom optimizer with weight decay
optimizer = optim.Adam(Unet_model.parameters(), lr=0.001)

Unet_path = './Unet_DiceScore.pt'
train_losses, valid_losses, train_DSC, train_IoU, train_recall, train_precision,valid_DSC, valid_IoU, valid_recall, valid_precision, best_DSC = train(Unet_model,train_loader, valid_loader, loss_fn, optimizer, Evaluation, epochs=30, save_path = Unet_path)

In [ ]:
plot_accuracies(train_DSC, valid_DSC)
plt.savefig('./Dice_plot_Unet.png')

In [ ]:
plot_losses(train_losses, valid_losses)
plt.savefig('./Losses_plot_Unet.png')